In [12]:
import os
from snowflake.snowpark import Session
import pandas as pd
import requests

In [2]:
#Connectig to snowflake
connection_parameters = {
"account": "ld77469.uae-north.azure",
"user": "GOKS98",
"password": "March20311",
"role": "ACCOUNTADMIN",
"warehouse": "SNOWFLAKE_WAREHOUSE"
 }

session = Session.builder.configs(connection_parameters).create()

In [3]:
#Creating a Dataframe
df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE").select(["W_WAREHOUSE_SK", "W_WAREHOUSE_NAME", "W_STATE"])
df_table.show()

-------------------------------------------------------
|"W_WAREHOUSE_SK"  |"W_WAREHOUSE_NAME"    |"W_STATE"  |
-------------------------------------------------------
|1                 |Conventional childr   |LA         |
|2                 |Important issues liv  |SD         |
|3                 |Doors canno           |WV         |
|4                 |Bad cards must make.  |NE         |
|5                 |NULL                  |CO         |
|6                 |Local, mass universi  |WV         |
|7                 |Quite effectiv        |WV         |
|8                 |Plain, reluctant      |MI         |
|9                 |Rooms cook            |TN         |
|10                |National, comple      |WV         |
-------------------------------------------------------



In [4]:
#Checking the columns
df_table_sql = session.sql("""SELECT W_STATE, COUNT(W_WAREHOUSE_NAME) FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE
                              GROUP BY W_STATE
                              ORDER BY 2 DESC""")
df_table_sql.show()


-----------------------------------------
|"W_STATE"  |"COUNT(W_WAREHOUSE_NAME)"  |
-----------------------------------------
|CO         |4                          |
|WV         |4                          |
|LA         |2                          |
|SD         |2                          |
|NC         |2                          |
|MO         |2                          |
|NE         |1                          |
|MI         |1                          |
|TN         |1                          |
|NM         |1                          |
-----------------------------------------



In [5]:
#Joining tables
df_1 = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.INVENTORY").select(["INV_WAREHOUSE_SK","INV_ITEM_SK", "INV_QUANTITY_ON_HAND"])
df_join = df_table.join(df_1, df_1["INV_WAREHOUSE_SK"] == df_table["W_WAREHOUSE_SK"]).select(["W_WAREHOUSE_SK", "W_WAREHOUSE_NAME", "W_STATE", "INV_ITEM_SK",\
                        "INV_QUANTITY_ON_HAND"])
df_join.show()

------------------------------------------------------------------------------------------------
|"W_WAREHOUSE_SK"  |"W_WAREHOUSE_NAME"    |"W_STATE"  |"INV_ITEM_SK"  |"INV_QUANTITY_ON_HAND"  |
------------------------------------------------------------------------------------------------
|25                |Unexpected sup        |NC         |161206         |529                     |
|2                 |Important issues liv  |SD         |161206         |921                     |
|16                |Governments should    |CO         |161206         |871                     |
|23                |Total sports leap a   |CO         |161206         |889                     |
|18                |Surprising emot       |FL         |161206         |60                      |
|9                 |Rooms cook            |TN         |161206         |862                     |
|10                |National, comple      |WV         |161206         |649                     |
|24                |Soon regul

In [6]:
df_join.count()         #Checking if the join was correct

1311525000

In [13]:
response = requests.get("https://api.openweathermap.org/data/2.5/weather?lat=25.2048&lon=55.2708&appid=b3effbd4d36024a4b9735d33830e7ca3")
data = response.json()
print(data)

{'coord': {'lon': 55.2708, 'lat': 25.2048}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 305.17, 'feels_like': 305.46, 'temp_min': 305.16, 'temp_max': 308.36, 'pressure': 1013, 'humidity': 40}, 'visibility': 7000, 'wind': {'speed': 4.63, 'deg': 330}, 'clouds': {'all': 28}, 'dt': 1667303711, 'sys': {'type': 1, 'id': 7537, 'country': 'AE', 'sunrise': 1667269578, 'sunset': 1667309914}, 'timezone': 14400, 'id': 290845, 'name': 'Ash Shindaghah', 'cod': 200}


In [14]:
weather_df = pd.json_normalize(data)
weather_df

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.pressure,main.humidity,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,7000,1667303711,14400,290845,Ash Shindaghah,200,55.2708,25.2048,...,1013,40,4.63,330,28,1,7537,AE,1667269578,1667309914
